In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [26]:
df = pd.read_csv("./02_Test_Data_Set/edu_enrollees.csv")
df_pre1 = df.drop(columns=["city", "company_size", "company_type"]).copy()
df_pre2 = df_pre1.dropna()

In [28]:
df_pre3 = df_pre2.loc[~df_pre1["experience"].isin([">20", "<1"])]
df_pre3["experience"] = df_pre3["experience"].astype(int)
df_pre3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9653 entries, 1 to 19154
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             9653 non-null   float64
 1   city_development_index  9653 non-null   float64
 2   gender                  9653 non-null   object 
 3   relevant_experience     9653 non-null   object 
 4   enrolled_university     9653 non-null   object 
 5   education_level         9653 non-null   object 
 6   major_discipline        9653 non-null   object 
 7   experience              9653 non-null   int32  
 8   last_new_job            9653 non-null   object 
 9   training_hours          9653 non-null   float64
 10  target                  9653 non-null   float64
 11  Xgrp                    9653 non-null   object 
dtypes: float64(4), int32(1), object(7)
memory usage: 942.7+ KB


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26500\958551526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre3["experience"] = df_pre3["experience"].astype(int)


In [31]:
df_pre4 = df_pre3.loc[~df_pre1["last_new_job"].isin([">4", "never"])]
df_pre4["last_new_job"] = df_pre4["last_new_job"].astype(int)
df_base = df_pre4.copy()
df_base

C:\Users\ASUS\AppData\Local\Temp\ipykernel_26500\1153953151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre4["last_new_job"] = df_pre4["last_new_job"].astype(int)


enrollee_id  city_development_index  gender      relevant_experience  \
8          27107.0                   0.920    Male  Has relevant experience   
11         23853.0                   0.920    Male  Has relevant experience   
19         11399.0                   0.827  Female  Has relevant experience   
20         31972.0                   0.843    Male  Has relevant experience   
21         19061.0                   0.926    Male  Has relevant experience   
...            ...                     ...     ...                      ...   
19149        251.0                   0.920    Male  Has relevant experience   
19150      32313.0                   0.920  Female  Has relevant experience   
19152      29754.0                   0.920  Female  Has relevant experience   
19153       7386.0                   0.878    Male   No relevant experience   
19154      31398.0                   0.920    Male  Has relevant experience   

      enrolled_university education_level major_discipline  experience  \
8           no_enrollment        Graduate             STEM           7   
11          no_enrollment        Graduate             STEM           5   
19          no_enrollment        Graduate             Arts           4   
20          no_enrollment         Masters             STEM          11   
21          no_enrollment         Masters             STEM          11   
...                   ...             ...              ...         ...   
19149       no_enrollment         Masters             STEM           9   
19150       no_enrollment        Graduate             STEM          10   
19152       no_enrollment        Graduate       Humanities           7   
19153       no_enrollment        Graduate       Humanities          14   
19154       no_enrollment        Graduate             STEM          14   

       last_new_job  training_hours  target   Xgrp  
8                 1            46.0     1.0  train  
11                1           108.0     0.0  train  
19                1           132.0     1.0  train  
20                1            68.0     0.0  train  
21                2            50.0     0.0  train  
...             ...             ...     ...    ...  
19149             1            36.0     1.0   test  
19150             3            23.0     0.0   test  
19152             1            25.0     0.0   test  
19153             1            42.0     1.0   test  
19154             4            52.0     1.0   test  

[7522 rows x 12 columns]

In [35]:
df_base["target"].unique()

array([1., 0.])

In [33]:
df_base.groupby("relevant_experience")["target"].count()

relevant_experience
Has relevant experience    6109
No relevant experience     1413
Name: target, dtype: int64

In [34]:
df_base.groupby("relevant_experience")["target"].sum()

relevant_experience
Has relevant experience    1319.0
No relevant experience      541.0
Name: target, dtype: float64

In [37]:
round((541/1413)/(1319/6109),2)

1.77

array(['Male', 'Female', 'Other'], dtype=object)

In [110]:
df_dummies = pd.get_dummies(data = df_base,columns=["gender", "relevant_experience", "enrolled_university", "education_level", "major_discipline"], dtype = "int")
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7522 entries, 8 to 19154
Data columns (total 24 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   enrollee_id                                  7522 non-null   float64
 1   city_development_index                       7522 non-null   float64
 2   experience                                   7522 non-null   int32  
 3   last_new_job                                 7522 non-null   int32  
 4   training_hours                               7522 non-null   float64
 5   target                                       7522 non-null   float64
 6   Xgrp                                         7522 non-null   object 
 7   gender_Female                                7522 non-null   int32  
 8   gender_Male                                  7522 non-null   int32  
 9   gender_Other                                 7522 non-null   int32  
 10  rele

In [111]:
df_dummies = df_dummies.drop(columns=["gender_Other", "relevant_experience_No relevant experience", "enrolled_university_no_enrollment", "education_level_Phd", "major_discipline_STEM"])

In [112]:
df_front = df_base[["target", "Xgrp", "city_development_index", "experience", "last_new_job", "training_hours"]]

In [113]:
df_back = df_dummies.loc[:, "gender_Female":]

In [114]:
job2 = pd.concat([df_front, df_back], axis=1)
job2.shape

(7522, 18)

In [115]:
model = LogisticRegression(random_state = 123, C = 100000, max_iter = 1000, solver = "liblinear", fit_intercept=True)

In [116]:
result = model.fit(X = df_p2.iloc[:, 2:], y = df_p2["target"])

In [118]:
np.exp(result.coef_).max()

1.6710665518951717

In [121]:
df_train = job2.loc[job2["Xgrp"] == "train"]
df_test = job2.loc[job2["Xgrp"] == "test"]
len(df_train), len(df_test)

(4706, 2816)

In [122]:
model = KNeighborsClassifier(n_neighbors = 5).fit(X = df_train.iloc[:,2:], y = df_train["target"])

In [123]:
result = model.predict(df_test.iloc[:, 2:])
result

array([0., 0., 0., ..., 0., 0., 0.])

In [124]:
df_test["target"]

12001    0.0
12002    1.0
12003    0.0
12004    0.0
12008    0.0
        ... 
19149    1.0
19150    0.0
19152    0.0
19153    1.0
19154    1.0
Name: target, Length: 2816, dtype: float64

In [125]:
y_t = df_test["target"]
y_p = result

In [126]:
df_tab = pd.crosstab(y_t, y_p)
df_tab

col_0    0.0  1.0
target           
0.0     1900  192
1.0      614  110